In [ ]:
from albumentations import (
    Compose, HorizontalFlip, VerticalFlip, Rotate,
    RandomBrightnessContrast, HueSaturationValue,
    ShiftScaleRotate, RandomResizedCrop, RandomBrightnessContrast,
    MotionBlur, MedianBlur, GaussianBlur, GaussNoise, 
    ElasticTransform, GridDistortion, OpticalDistortion,
    Cutout, CoarseDropout, Blur, RGBShift, 
    CLAHE, ChannelShuffle, ToGray
)
import cv2
import numpy as np

# Define the augmentation pipeline
augmentation = Compose([
    HorizontalFlip(p=0.5),
#    VerticalFlip(p=0.5),
  Rotate(limit=20, p=1),
     RandomBrightnessContrast(p=0.3),
#     HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1),
#     ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.5),
#        RandomResizedCrop(height=120, width=160, scale=(0.3, 0.5), p=1),
#     MotionBlur(blur_limit=(15, 31), p=1),
#     MedianBlur(blur_limit=(5, 7), p=1),
#     GaussianBlur(blur_limit=(3, 7), p=1),
#     GaussNoise(var_limit=(100.0, 500.0), p=1),
#    ElasticTransform(alpha=1, sigma=10, alpha_affine=10, p=1),
#     GridDistortion(num_steps=5, distort_limit=0.3, p=1), #not imp
#     OpticalDistortion(distort_limit=0.3, shift_limit=0.3, p=1), # not imp
#     Cutout(num_holes=8, max_h_size=16, max_w_size=16, p=1),
#    CoarseDropout(max_holes=8, max_height=16, max_width=16, p=1), #imp
#     Blur(blur_limit=3, p=1), # Not imp gaussian blur will be enough
#     RGBShift(r_shift_limit=30, g_shift_limit=30, b_shift_limit=30, p=1) # not imp\
 #   Shear(shar_limit=0.5, p=1),
])




from matplotlib import pyplot as plt

for i in range(100):
    img_list = []

    # Load an image
    image = cv2.imread("/home/aman/ai/single_img_data_training/train/none/16873233598408537.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    img_list.append(image)

    # Apply the augmentation
    augmented_image = augmentation(image=image)['image']
    img_list.append(augmented_image)

    final_img = img_list[0]

    for img in img_list[1:]:
        final_img = np.concatenate((final_img, img), axis=1)    
    plt.imshow(final_img)
    plt.show()



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential

import numpy as np
# !pip install numpy==1.20
import matplotlib.pyplot as plt
# !pip install autokeras
# import autokeras as ak
import glob
import os, cv2

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications import MobileNetV3Large

from skimage.util import random_noise

In [ ]:

BATCH_SIZE=3

IMAGE_SIZE_WIDTH= 80
IMAGE_SIZE_HEIGHT = 60 
NUM_CHANNEL = 3                                   

IMAGE_SHAPE = (IMAGE_SIZE_HEIGHT, IMAGE_SIZE_WIDTH) #(height, width)
MODEL_IMAGE_SHAPE = (IMAGE_SIZE_HEIGHT, IMAGE_SIZE_WIDTH, NUM_CHANNEL) 

In [ ]:
train_data_dir='/home/aman/ai/single_img_data_training/train'
val_data_dir='/home/aman/ai/single_img_data_training/val'

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator





# from tensorflow.keras.layers.preprocessing.image_preprocessing import HORIZONTAL

train_datagen=ImageDataGenerator(   rotation_range=15,       # Rotate images randomly up to 20 degrees
                                    width_shift_range=0.15,   # Shift the width by a fraction of the total width
                                    height_shift_range=0.15,  # Shift the height by a fraction of the total height
                                    zoom_range=0.15,# Randomly zoom into images by up to 20%
                                    shear_range=0.15, 
#                                     zca_whitening= True,
                                    brightness_range=[0.9, 1.35], 
                                 )

val_datagen=ImageDataGenerator(#rescale=1./255
                               )




train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=IMAGE_SHAPE, # accept height and wdith order
                                                  batch_size=BATCH_SIZE,
#                                                   color_mode="grayscale",
                                                  class_mode='sparse',
                                                  
                                                 )

val_generator=val_datagen.flow_from_directory(val_data_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
#                                               color_mode="grayscale",
                                              class_mode='sparse')






In [ ]:
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=IMAGE_SHAPE, # accept height and wdith order
                                                  batch_size=16,
#                                                   color_mode="grayscale",
                                                  class_mode='sparse',
                                                  
                                                 )
def train_gen():

    
    for train_x, train_y in train_generator:
        for i in range(train_x.shape[0]):
            #print(train_x[0].shape)
            train_x[i] = augmentation(image=train_x[i])["image"]
            #pass
        
        #print("yeild")
        yield train_x, train_y    

In [ ]:
class_names = list(train_generator.class_indices.keys())
NUM_CLASSES = len(class_names)
print("Class Names : ", class_names)

In [ ]:
class_names_instances = {}
max_instances = 0
for i, folder_path in enumerate(sorted(glob.glob(train_data_dir+"/*"))):

    class_names_instances[folder_path.split("/")[-1]] = len(os.listdir(folder_path))
    if max_instances < len(os.listdir(folder_path)):
        max_instances = len(os.listdir(folder_path))
    

In [ ]:
class_names_instances

In [ ]:
class_weights = {}


for i, class_name in enumerate(class_names):
    class_weights[i] = max_instances/class_names_instances[class_name]



for i, class_name in enumerate(class_names):
    print(f"{i} : {class_name}")
    
print(" ")

print("class_weights : ",class_weights)

In [ ]:
images, labels = next(train_gen())
batch_size =5
fig, axes = plt.subplots(nrows=1, ncols=batch_size, figsize=(10, 4))

for i in range(batch_size):
    
    # Rescale pixel values to [0, 1]
    image = images[i] / 255.0
    
    axes[i].imshow(image)
    axes[i].axis('off')
    axes[i].set_title('{}'.format(class_names[int(labels[i])]))

plt.tight_layout()
plt.show()



In [ ]:

from tensorflow.keras.applications import MobileNetV3Small
num_classes = 3

base_model = MobileNetV3Small(
                                input_shape=(60, 80, 3),
                                alpha=0.01,
                                dropout_rate=0.2,
                                minimalistic=True, # if True, aims to provide a lightweight and efficient model for image classification tasks with minimal computational resources.
                                 
                                include_top=False,
                                weights=None , #'imagenet' , None
                                
                             )
base_model.trainable = True

print(base_model.summary())

model = tf.keras.Sequential([
    base_model,
#     layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
#     layers.Dense(16, activation='relu'),
#     layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


model.summary()

In [ ]:
from tensorflow.keras import Model

# Get the number of layers in the base model
num_layers = len(base_model.layers)

# Create a new model without the last 10 layers
new_base_model = Model(base_model.input, base_model.layers[num_layers-75-1].output)

new_base_model.trainable = True

# Print summary
print(new_base_model.summary())

model = tf.keras.Sequential([
    new_base_model,
    Conv2D(3, (3, 3), strides=(2, 2), padding='valid', use_bias=True),
#     layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
#     layers.Dense(16, activation='relu'),
#     layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("image_classification_checkpoint.h5",
                             monitor='val_sparse_categorical_accuracy',
                             mode='max',
                             save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor='val_accuracy',
                          min_delta=0,
                          patience=5,
                          verbose=1,
                          restore_best_weights=True)

# callbacks=[checkpoint,earlystop]
callbacks=[checkpoint]
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                   optimizer=Adam(learning_rate=0.001),
                   metrics=['sparse_categorical_accuracy'])

epochs=100

batch_size = 16

steps_per_epoch = len(train_generator)

history = model.fit(     train_gen(),
                        #  steps_per_epoch=num_train_samples//batch_size,
                         epochs=epochs,
                        #validation_split=0.15,
                        callbacks=callbacks,
                         validation_data=val_generator,
                        #  validation_steps=num_val_samples//batch_size
                        #class_weight=class_weights,
                    steps_per_epoch=steps_per_epoch
                    )

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(base_model, to_file='model.png', show_shapes=True, show_layer_names=True)

# Display the image in jupyter
Image(filename='model.png')

In [ ]:
from tensorflow.keras.utils import plot_model
from PIL import Image

# Generate the plot
plot_model(base_model, to_file='model.png', show_shapes=True, show_layer_names=True, dpi=300)

# Open the image using PIL
im = Image.open('model.png')

# Specify a new size
new_size = (im.size[0]//3, im.size[1]//3)  # for example, to reduce size by half

# Resize the image
im.thumbnail(new_size, Image.ANTIALIAS)

# Display the image
im.show()